## Pandas Split/Apply/Combine Functionality

In this tutorial, we will review how to operate on different portions of your data in different ways. We will also learn how to use more advanced functionality straight in Pandas split/apply/combine api to perform operations. 

The tutorial will follow:

* [Ingesting Data](#ingest)
* [Basic Split/Apply/Combine](#basic)
* [Enhanced Split/Apply/Combine Features](#enhanced)

In [1]:
import pandas as pd
import requests
import io

### Ingesting Data <a id='ingest' > </a>

To keep the tutorial self contained, we will pull our data straight from the web. We will be using the [UCI Machine Learning wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality). 

In [2]:
# UCI machine learning has a white wine and red wine quality dataset
red_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
white_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

# use requests to secure the data
red_raw = requests.get(red_url).content
white_raw = requests.get(white_url).content

In [3]:
# we can use the io package to decode the data and feed into pandas.
# A good discussion on this can be located at: https://stackoverflow.com/questions/32400867/pandas-read-csv-from-url

red_df = pd.read_csv(io.StringIO(red_raw.decode('utf-8')), sep=';')
white_df = pd.read_csv(io.StringIO(white_raw.decode('utf-8')), sep=';')

# assign the type of wine
red_df['Type'] = 'Red'
white_df['Type'] = 'White'

# and finally merge into a single dataset
df = pd.concat([red_df, white_df])

assert df.shape == (6497, 13), 'Incorrect number of rows obtains'

In [4]:
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,Red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,Red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,Red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,Red
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,Red
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,Red
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,Red
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,Red


In [5]:
# we will bin the alcohol content into low, medium, high to further illustrate pandas split/apply/combine functionality
df['alcohol'].describe()

count    6497.000000
mean       10.491801
std         1.192712
min         8.000000
25%         9.500000
50%        10.300000
75%        11.300000
max        14.900000
Name: alcohol, dtype: float64

In [6]:
# create the three bins
df['alcoholbins'] = pd.cut(df['alcohol'], 3, labels=['low', 'medium', 'high'])

### Basic split/apply/combine functionality <a id='basic' ></a>

Based on the bins above, we may be interested in understanding what is the average alcohol content across low, medium, and high. We can start to use pandas built in functionality for this. 

In [7]:
# get the mean alcohol content for low, medium, high bins
df.groupby('alcoholbins')['alcohol'].mean()

alcoholbins
low        9.514633
medium    11.302820
high      13.052491
Name: alcohol, dtype: float64

In [8]:
# and lets see how many observations there are under each
df.groupby('alcoholbins')['alcohol'].count()

alcoholbins
low       3295
medium    2846
high       356
Name: alcohol, dtype: int64

In [9]:
# you can also iterate over a groupby in a regular for loop
for name, group in df.groupby('alcoholbins'):
    print(name)

low
medium
high


In [10]:
# and to see further what is happening, each group assigned to the group variable in the for loop 
# is your actual data for that group. You can see this by peeking into the groups:

for name, group in df.groupby('alcoholbins'):
    print('---------------')
    print('Current group: {}'.format(name))
    print(group.head(2))

---------------
Current group: low
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70          0.0             1.9      0.076   
1            7.8              0.88          0.0             2.6      0.098   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   

   alcohol  quality Type alcoholbins  
0      9.4        5  Red         low  
1      9.8        5  Red         low  
---------------
Current group: medium
    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
9             7.5               0.5         0.36             6.1      0.071   
11            7.5               0.5         0.36             6.1      0.071   

    free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
9                  17.0                

In [11]:
# occasionally, you will want to derive new features using aggregate values for a group. In order to assign these
# values back to the original dataframe, we can reset the index and merge back in using the following:

# create a copy of the dataframe
dfcopy = df.copy(deep=True)

# get the counts by group, reset the idnex, and give the column a more meaningful name
alc_counts = df.groupby('alcoholbins')['alcohol'].count().reset_index().rename(columns={'alcohol': 'alcoholcount'})

# merge back in with original data
dfcopy = pd.merge(dfcopy, alc_counts, on='alcoholbins', how='inner')

# and now we have assigned the total observations per alcohol bin to all rows of the dataframe
dfcopy.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type,alcoholbins,alcoholcount
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red,low,3295
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,Red,low,3295
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,Red,low,3295
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,Red,low,3295
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red,low,3295


In [12]:
# to unpack the groupby mean, it is the same as doing the following except for all groups

df[df['alcoholbins'] == 'low'].loc[:, 'alcohol'].mean() # which matches our original groupby value for the low bin

9.514633282751674

In [13]:
# now what if we were interested in the average alcohol content by alcohol bin and by type of wine?
# simply add the additional groupby variable
df.groupby(['Type', 'alcoholbins'])['alcohol'].mean()

Type   alcoholbins
Red    low             9.621848
       medium         11.223283
       high           13.122917
White  low             9.475868
       medium         11.326834
       high           13.037055
Name: alcohol, dtype: float64

In [14]:
# this is the same as the following
df[(df['Type'] == 'Red') & (df['alcoholbins'] == 'low')].loc[:, 'alcohol'].mean()

9.621847619047628

In [15]:
# sometimes we may be interested in different types of aggregation methods based on the columns.
# for instance, if we want to find the average chlorides by group by the standard deviation for volatile acidity
# we can use:

res = df.groupby(['Type', 'alcoholbins']).agg({'chlorides': 'mean',
                                        'volatile acidity': 'std'}).reset_index().rename(columns={'chlorides': 'chl_mean',
                                                                                         'volatile acidity': 'acid_std'})

res.head()

,Type,alcoholbins,chl_mean,acid_std
0,Red,low,0.095206,0.164005
1,Red,medium,0.079427,0.191759
2,Red,high,0.064563,0.157561
3,White,low,0.053418,0.102411
4,White,medium,0.038816,0.093599


In [16]:
# and to merge back in with the original data
dfcopy = pd.merge(dfcopy, res, on=['Type', 'alcoholbins'], how='inner')

dfcopy.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type,alcoholbins,alcoholcount,chl_mean,acid_std
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red,low,3295,0.095206,0.164005
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,Red,low,3295,0.095206,0.164005
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,Red,low,3295,0.095206,0.164005
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,Red,low,3295,0.095206,0.164005
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red,low,3295,0.095206,0.164005
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,Red,low,3295,0.095206,0.164005
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,Red,low,3295,0.095206,0.164005
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,Red,low,3295,0.095206,0.164005
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,Red,low,3295,0.095206,0.164005
9,6.7,0.58,0.08,1.8,0.097,15.0,65.0,0.9959,3.28,0.54,9.2,5,Red,low,3295,0.095206,0.164005


In [17]:
# we can also retrieve specific groups after a groupby has been applied

groups = df.groupby(['Type', 'alcoholbins']).get_group(('White', 'low'))

groups.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type,alcoholbins
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,White,low
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,White,low
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,White,low
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,White,low
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,White,low


### Enhanced Split/Apply/Combine Functionality <a id='enhanced' ></a>

Using basic pandas functionality is great, however it can only get you so far. Sometime we may want to perform very specific operations on groups of our data. To start with custom functions, we can do the easiest case where there is only one parameter to the function, which is the group itself. 

In [18]:
def custom_group_func(group):
    """
    perform custom operation on group of data. Here we will tag quality groups defined as average quality
    score above 9
    """
    group_mean = group['quality'].mean()
    if group_mean > 6:
        group['Quals'] = 'High Quality Group'
    else:
        group['Quals'] = 'Regular Quality Group'
    return group.reset_index(drop=True)

In [19]:
res = pd.concat([df.groupby(['Type', 'alcoholbins']).apply(custom_group_func)]).reset_index(drop=True)
res.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type,alcoholbins,Quals
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red,low,Regular Quality Group
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,Red,low,Regular Quality Group
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,Red,low,Regular Quality Group
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,Red,low,Regular Quality Group
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Red,low,Regular Quality Group


In [20]:
# So what ar ethe high quality groups?
res[res['Quals'].str.contains('High')][['Type', 'alcoholbins']].drop_duplicates()
# Red Wine with high alcohol
# white wine with medium alcohol
# White wine with high alcohol
# seems as if people really enjoy their high alcohol content

,Type,alcoholbins
1535,Red,high
4019,White,medium
6205,White,high


In [21]:
# just out of pure curiousity, we can see the average quality by alcohol content
qual_means_groups = df.groupby('alcoholbins')['quality'].mean()
qual_mean = df['quality'].mean()
std = df['quality'].std()
print("""Average Quality: {}
        \n--------------\nQuality Groups Standard Deviation: {}
        \n--------------\n{}""".format(qual_mean, std, qual_means_groups))

# wines with high alcohol content are nearly a full standard deviation away from the average quality across all wines
# people definitely enjoy their alcohol content :) 

Average Quality: 5.818377712790519
        
--------------
Quality Groups Standard Deviation: 0.873255271531111
        
--------------
alcoholbins
low       5.484674
medium    6.099789
high      6.657303
Name: quality, dtype: float64


Our custom function applied to pandas groupby could only handle 1 parameter. What if we wanted more?
Pandas allows you to feed in multiple parameters into an apply statement, however the syntax is a bit different
than one might originally expect. 

Going back to the wine quality and alcohol content example, what if we wanted to flag all groups that are at least half a standard deviation away from the population mean? 

In [22]:
def flag_n_std(group, 
                std_flag=2, 
                pop_mean=None,
                pop_std=None,
                col_of_interest='quality',
                flag='STD_FLAG'):
    """
    flag any groups more than std_flag above the population mean
    
    params
    -------
    group --> group of data being operated on
    std_flag --> number of standard deviations away 
    col_of_interest --> Column we want to flag as being std_flag away from the population mean
    pop_mean --> population mean for column
    pop_std --> population standard deviation
    flag --> value to fill in as our flag
    """
    # get group mean
    group_mean = group.loc[:, col_of_interest].mean()
    # get the threshold
    thresh = std_flag * pop_std
    # get the difference
    diff = group_mean - pop_mean
    # check
    if diff > thresh:
        group['STD_FLAG'] = flag
    else:
        group['STD_FLAG'] = None
    return group.reset_index()

In [23]:
# set some basic parameter values
col_of_interest = 'quality'
pop_mean = df[col_of_interest].mean()
pop_std = df[col_of_interest].std()
flag = 'GREAT_QUALITY'

In [24]:
# apply partial function to dataframe
res = pd.concat([df.groupby(['Type', 'alcoholbins']).apply(flag_n_std, std_flag=0.5,
                                                          pop_mean=pop_mean,
                                                          pop_std=pop_std,
                                                          flag=flag).reset_index(drop=True)])

res[res['STD_FLAG'].notnull()][['Type', 'alcoholbins']].drop_duplicates()
# across both red and white wines, high alcohol content translates into better quality :) 

,Type,alcoholbins
1535,Red,high
6205,White,high


And thats a wrap. Pandas split/apply/combine can be very powerful. It can support your base aggregation methods all the way up to more advanced custom functions. The key thing to remember is all it is doing is operating on a specific portion of your data, where the groupby columns equal specific combinations of values. The full code for this tutorial can be downloaded [here](https://github.com/datawrestler/Tutorials/tree/master/Pandas)